In [1]:
import cv2
import os

face_cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(face_cascade_path)

if face_cascade.empty():
    raise IOError("Failed to load Haar cascade file.")

def detect_mask(frame, face_cascade):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(60, 60), flags=cv2.CASCADE_SCALE_IMAGE)
    
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        
        mask_region = face[int(h/2):h, 0:w]

        hsv = cv2.cvtColor(mask_region, cv2.COLOR_BGR2HSV)
        
        lower_color = (100, 100, 50)   
        upper_color = (140, 255, 255)
        
        mask = cv2.inRange(hsv, lower_color, upper_color)
        
        mask_percentage = (cv2.countNonZero(mask) / (w * int(h/2))) * 100

        label = "Mask" if mask_percentage > 50 else "No Mask"  
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

    return frame

print("[INFO] Starting video stream...")
vs = cv2.VideoCapture(0)

while True:

    ret, frame = vs.read()
    
    if not ret:
        break

    frame = detect_mask(frame, face_cascade)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord("q"):
        break

cv2.destroyAllWindows()
vs.release()


[INFO] Starting video stream...
